# 오픈 포즈 학습

In [1]:
# 패키지 import
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 초기 설정
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 데이터 로더 및 네트워크 작성

In [2]:
from utils.dataloader import *

train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(rootpath="./data/")

train_dataset = COCOkeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())
# val_dataset = COCOkeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

batch_size = 32

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train" : train_dataloader, "val" : None}

In [3]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()

## 손실함수 정의

In [4]:
class OpenPoseLoss(nn.Module):
    def __init__(self):
        super(OpenPoseLoss, self).__init__()

    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        total_loss = 0

        for j in range(6):
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask

            pred2 = saved_for_loss[2*j + 1] * heat_mask
            gt2 = heatmap_target.float() * heat_mask

            total_loss += F.mse_loss(pred1, gt1, reduction="mean") + F.mse_loss(pred2, gt2, reduction="mean")

        return total_loss

criterion = OpenPoseLoss()

## 학습 실시

In [5]:
optimizer = optim.SGD(net.parameters(),lr=1e-2, momentum=0.9, weight_decay=0.0001)

In [6]:
# 모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 화상의 매수
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # 반복 카운터 설정
    iteration = 1

    # epoch 루프
    for epoch in range(num_epochs):

        # 개시 시간을 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
                optimizer.zero_grad()
                print('(train)')

            # 이번에는 검증을 생략
            else:
                continue
                # net.eval()   # 모델을 검증 모드로
                # print('-------------')
                # print('(val)')

            # 데이터 로더에서 minibatch씩 꺼내는 루프
            for imges, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # 미니 배치 크기가 1이면, 배치 노멀라이제이션에서 에러가 발생하므로 피한다
                if imges.size()[0] == 1:
                    continue

                # GPU가 사용 가능하면 GPU로 데이터를 보낸다
                imges = imges.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    # (out6_1, out6_2)는 사용하지 않으므로 _ 로 대체
                    _, saved_for_loss = net(imges)

                    loss = criterion(saved_for_loss, heatmap_target,
                                     heat_mask, paf_target, paf_mask)
                    del saved_for_loss
                    # 훈련시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iter에 1번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 검증시
                    # else:
                        #epoch_val_loss += loss.item()

        # epoch의 phase별 loss와 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    # 마지막 네트워크를 저장한다
    torch.save(net.state_dict(), 'weights/openpose_net_' +
               str(epoch+1) + '.pth')


In [7]:
# 학습 및 검증을 실행한다
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-------------
Epoch 1/2
-------------
(train)


/home/plass-heesu/.conda/envs/tf2.5/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


반복 10 || Loss: 0.0093 || 10iter: 26.2554 sec.
반복 20 || Loss: 0.0082 || 10iter: 20.3005 sec.
반복 30 || Loss: 0.0068 || 10iter: 19.7967 sec.
반복 40 || Loss: 0.0057 || 10iter: 19.7051 sec.
반복 50 || Loss: 0.0048 || 10iter: 19.5485 sec.
반복 60 || Loss: 0.0044 || 10iter: 19.8280 sec.
반복 70 || Loss: 0.0037 || 10iter: 20.1313 sec.
반복 80 || Loss: 0.0030 || 10iter: 20.0685 sec.
반복 90 || Loss: 0.0027 || 10iter: 19.5284 sec.
반복 100 || Loss: 0.0026 || 10iter: 20.0230 sec.
반복 110 || Loss: 0.0021 || 10iter: 19.6482 sec.
반복 120 || Loss: 0.0022 || 10iter: 19.9070 sec.
반복 130 || Loss: 0.0020 || 10iter: 19.8089 sec.
반복 140 || Loss: 0.0018 || 10iter: 19.4712 sec.
반복 150 || Loss: 0.0019 || 10iter: 19.7283 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:0.0043 ||Epoch_VAL_Loss:0.0000
timer:  315.9997 sec.
-------------
Epoch 2/2
-------------
(train)
반복 160 || Loss: 0.0017 || 10iter: 13.8279 sec.
반복 170 || Loss: 0.0016 || 10iter: 19.7668 sec.
반복 180 || Loss: 0.0020 || 10iter: 20.7655 sec.
반복 190 || Loss: 0.0016